In [1]:
# Standard library imports
import os
import sys
import copy
from typing import Union, Optional

# Third party imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import empyrical as ep

# Add the project root directory to Python path
project_root = os.path.dirname(os.path.dirname(os.getcwd()))   #<Change this path if needed>
src_path = os.path.join(project_root, 'qpmwp-course\\src')    #<Change this path if needed>
sys.path.append(project_root)
sys.path.append(src_path)

# Local modules imports
from helper_functions import load_data_spi, load_pickle
from estimation.covariance import Covariance
from estimation.expected_return import ExpectedReturn
from optimization.optimization import Optimization, Objective, MeanVariance
from optimization.optimization_data import OptimizationData
from optimization.constraints import Constraints
from backtesting.backtest_item_builder_classes import (
    SelectionItemBuilder,
    OptimizationItemBuilder,
)
from backtesting.backtest_item_builder_functions import (
    bibfn_selection_min_volume,
    bibfn_selection_gaps,
    bibfn_return_series,
    bibfn_budget_constraint,
    bibfn_box_constraints,
    bibfn_size_dependent_upper_bounds,
)
from backtesting.backtest_data import BacktestData
from backtesting.backtest_service import BacktestService
from backtesting.backtest import Backtest

In [2]:
df = pd.read_csv('c:/Users/admin\OneDrive/Dokumente/Studium UZH/Master/5. Semester/Quantitative Portfolio Management with Python/Assingments/1. Assingment/2. Werkstatt/qpmwp-course/data/msci_country_indices.csv',
                 index_col=0, header=0, parse_dates=True, date_format='%d-%m-%Y')
#df = df.tail(2).iloc[:, :2]
print(df)

                  AT        AU        BE        CA        CH        DE  \
Index                                                                    
1999-01-01  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
1999-01-04  0.010057  0.009080  0.042147  0.013070  0.035885  0.052249   
1999-01-05  0.013661 -0.010048  0.020162  0.021940  0.012016  0.001444   
1999-01-06  0.000000  0.015264 -0.000078  0.027640  0.015335  0.036205   
1999-01-07  0.004104  0.016564 -0.016877 -0.003480 -0.011902 -0.020187   
...              ...       ...       ...       ...       ...       ...   
2023-04-12 -0.002488  0.004714 -0.012072  0.001708  0.000017  0.002662   
2023-04-13  0.001297 -0.003806 -0.000049  0.004728  0.000913  0.002400   
2023-04-14  0.009533  0.004829  0.007622  0.001950  0.008387  0.005797   
2023-04-17 -0.005665  0.003404  0.004917  0.003392 -0.001485 -0.002086   
2023-04-18  0.004836 -0.003262 -0.007127  0.002161  0.004373  0.006089   

                  DK        ES       

In [20]:
# Load data
path_to_data = "C:/Users/admin/OneDrive/Dokumente/Studium UZH/Master/5. Semester/Quantitative Portfolio Management with Python/Assingments/4. Assingment/1. Daten erhalten/" # <change this if necessary>

# Load market and jkp data from parquet files
market_data = pd.read_parquet(path = f'{path_to_data}market_data.parquet')

# Instantiate the BacktestData class
# and set the market data and jkp data as attributes
df1 = BacktestData()
df1.market_data = market_data
print(df1.market_data.head(10))
print(df1.market_data.tail(10))
returns = df1.get_return_series()
print(returns.head(20))
print(returns.tail(20))

               price        mktcap  liquidity sector
date       id                                       
1999-05-06 1   461.0  1.235952e+09   129080.0   None
1999-05-07 1   455.0  1.219866e+09     4550.0   None
1999-05-10 1   455.0  1.219866e+09      910.0   None
1999-05-11 1   460.0  1.233271e+09     4600.0   None
1999-05-12 1   460.0  1.233271e+09      460.0   None
1999-05-14 1   460.0  1.233271e+09     4600.0   None
1999-05-19 1   460.0  1.233271e+09     2300.0   None
1999-05-21 1   460.0  1.233271e+09     4600.0   None
1999-05-26 1   460.0  1.233271e+09    13800.0   None
1999-05-31 1   463.0  1.241314e+09    13890.0   None
                    price      mktcap  liquidity sector
date       id                                          
2024-04-17 313   9.814924  20167830.0        0.0   None
2024-04-18 313   9.814924  20167830.0        0.0   None
2024-04-19 313   9.814924  20167830.0        0.0   None
2024-04-22 313   9.814924  20167830.0        0.0   None
2024-04-23 313   9.814924  2

In [50]:
def mean_ewm(X: Union[pd.DataFrame, np.ndarray, object],
                    scalefactor: Union[float, int] = 1,
                    span: Union[float, int] = 10,
                    reverse: bool = True,
                    attribute: Optional[str] = None) -> Union[pd.Series, np.ndarray]: # False = starting from the top and true = starting from the bottom.

    if not isinstance(X, pd.DataFrame):
        if attribute is not None and hasattr(X, attribute) and isinstance(getattr(X, attribute), pd.DataFrame):
        # 1.) "hasattr(X, attribute)" checks whether the object X has an attribute with the name stored in the variable "attribute".
        # 2.) "isinstance(getattr(X, attribute), pd.DataFrame)" checks whether that the attribute of X is actually a pandas DataFrame, and not something else (like a string or list).
            X = X.get_return_series()
        else:
            raise ValueError("If X is a BacktestData object, an 'attribute' must be specified.")

    denominators_dict = {}
    numerators_dict = {}
    mu_dict = {}
    mu = []
    alpha = 2 / (span + 1)

    for col in X.columns:
        series = X[col].values
        if reverse:
            series = series[::-1]

        denominators = []
        numerators = []

        for t in range(len(series)):
            value = series[t]
            if pd.isna(value):
                continue  # "pd.isna(value)"" checks if the current value is missing (i.e., NaN, None, or pd.NA). If yes, "continue" tells Python to skip the rest of the loop body and go to the next iteration.
            weight = alpha * (1 - alpha) ** t
            numerators.append(weight * value)
            denominators.append(weight)

        # Store each list under the corresponding column name
        denominators_dict[col] = denominators
        numerators_dict[col] = numerators

    for col in X.columns:
        # print(f"Weights for {col}: {denominators_dict[col]}")
        # print(f"Weights * Value for {col}: {numerators_dict[col]}")
        mu = (np.sum(numerators_dict[col]) / np.sum(denominators_dict[col])) * scalefactor
        mu_dict[col] = mu

    return pd.Series(mu_dict, dtype = "float64")

In [52]:
span = 10
scalefactor = 1
reverse = True

test_mean_1 = mean_ewm(df, scalefactor, span, reverse, attribute = "market_data")
test_mean_2 = mean_ewm(df1, scalefactor, span, reverse, attribute = "market_data")
pd.set_option('display.max_rows', None) # Show all rows in print output
print("Test mean 1:")
print(test_mean_1)
print("Test mean 2:")
print(test_mean_2.sort_values(na_position='last', ascending=False))

Test mean 1:
AT    0.002798
AU    0.001788
BE    0.000074
CA    0.002864
CH    0.002667
DE    0.002657
DK    0.006635
ES    0.002692
FI   -0.000141
FR    0.003251
GB    0.003051
GR    0.005125
HK    0.001514
IE    0.005498
IL    0.000664
IT    0.003213
JP    0.003874
NL    0.000168
NO    0.003108
NZ   -0.000819
PT    0.000054
SE    0.003743
SG    0.000579
US    0.001963
dtype: float64
Test mean 2:
262     4.148447e-01
4       1.822691e-01
108     5.078812e-02
13      1.984882e-02
1       1.916220e-02
62      1.703043e-02
313     1.554145e-02
138     1.524175e-02
191     1.478527e-02
225     9.917392e-03
248     9.739873e-03
42      9.097819e-03
265     9.091967e-03
39      8.995017e-03
142     8.224370e-03
155     8.039672e-03
15      8.003513e-03
118     7.722644e-03
196     7.342027e-03
311     7.201185e-03
128     7.130020e-03
219     6.331015e-03
135     6.203915e-03
246     6.177279e-03
310     6.157604e-03
29      5.909603e-03
278     5.855489e-03
86      5.846777e-03
104     5.5

C:\Users\admin\AppData\Local\Temp\ipykernel_9132\468694968.py:44: RuntimeWarning: invalid value encountered in scalar divide
  mu = (np.sum(numerators_dict[col]) / np.sum(denominators_dict[col])) * scalefactor


In [29]:
attribute = "market_data"
if attribute is not None and hasattr(df1, attribute) and isinstance(getattr(df1, attribute), pd.DataFrame):
    X = df1.get_return_series()
else:
    raise ValueError("If X is a BacktestData object, an 'attribute' must be specified.")
print(X.head(20))
print(X.tail(20))

id           1  10  100  101  102  103  104  105  106  107  ...        90  91  \
date                                                        ...                 
1986-01-01 NaN NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...       NaN NaN   
1986-01-02 NaN NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...       NaN NaN   
1986-01-03 NaN NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...       NaN NaN   
1986-01-06 NaN NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ... -0.032000 NaN   
1986-01-07 NaN NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  0.008264 NaN   
1986-01-08 NaN NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ... -0.016393 NaN   
1986-01-09 NaN NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  0.000000 NaN   
1986-01-10 NaN NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ... -0.041667 NaN   
1986-01-13 NaN NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  0.013043 NaN   
1986-01-14 NaN NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ... -0.004292 NaN   
1986-01-15 NaN NaN  NaN  NaN